In [10]:
import json
import sys
from pathlib import Path
import ipywidgets as widgets
from IPython.display import display, clear_output
import matplotlib.pyplot as plt
import matplotlib.image as mpimg


In [11]:
PROJECT_ROOT = Path.cwd().parent
sys.path.insert(0, str(PROJECT_ROOT))

from pipeline.download_data import extract_game_ids

In [12]:
# Load game IDs for a specific team and season
game_ids = extract_game_ids("MTL", 2024)
print(len(game_ids))

87


In [13]:
# Load the json data for the season, stored in the data folder
data_path = Path("../data/nhl_play_by_play_MTL_2024_2025.json")
games = json.loads(data_path.read_text(encoding="utf-8"))

# Load the hockey rink image
rink_image_path = Path("../assets/hockey_rink.jpg")
rink_image = mpimg.imread(rink_image_path)

In [14]:
# Fonction to fetch a player's name by their ID
import requests

def normalize_player_id(x):
    """Retourne un int ou None, même si x est un dict/str/int."""
    if x is None:
        return None
    if isinstance(x, int):
        return x
    if isinstance(x, str):
        return int(x) if x.isdigit() else None
    if isinstance(x, dict):
        # cas fréquent: {"id": 8478402} ou {"playerId": 8478402}
        for k in ("id", "playerId", "scoringPlayerId"):
            if k in x:
                return normalize_player_id(x[k])
    return None


def get_player_name(player_id: int) -> str | None:
    url = f"https://api-web.nhle.com/v1/player/{player_id}/landing"
    r = requests.get(url, timeout=10)
    r.raise_for_status()
    data = r.json()

    first = data.get("firstName")
    last  = data.get("lastName")

    if isinstance(first, dict):
        first = first.get("default", "")
    if isinstance(last, dict):
        last = last.get("default", "")

    name = f"{first} {last}".strip()
    return name or None


# Cache for player names to avoid redundant API calls
player_cache = {}
def get_player_name_cached(player_id):
    pid = normalize_player_id(player_id)
    if pid is None:
        return None

    if pid in player_cache:
        return player_cache[pid]

    name = get_player_name(pid)
    player_cache[pid] = name
    return name


In [15]:
# Define functions to extract game labels, play coordinates, and play titles
def game_label(game):
    away_team = game.get("awayTeam", {}).get("abbrev", "AWAY")
    home_team = game.get("homeTeam", {}).get("abbrev", "HOME")
    away_score = game.get("awayTeam", {}).get("score", "?")
    home_score = game.get("homeTeam", {}).get("score", "?")
    game_date = game.get("gameDate", "????-??-??")
    game_id = game.get("id", "?")
    return f"{away_team} @ {home_team} | {away_score}-{home_score} | {game_date} | ID: {game_id}"

def play_coordinates(play):
    details = play.get("details") or {}
    if "xCoord" in details and "yCoord" in details:
        return float(details["xCoord"]), float(details["yCoord"])
    return None

def play_title(play):
    type = play.get("typeDescKey", "Unknown Play")
    period = play.get("periodDescriptor", {}).get("number", "?")
    time_in_period = play.get("timeInPeriod", "?")

    # if event is a goal, include scorer's name
    if type == "goal":
        scorer = get_player_name_cached(play.get("scoringPlayerId", {}))
        if scorer:
            type += f" by {scorer}"
            
    return f"{type} (Period {period}, Time {time_in_period})"

# Helper function to safely get nested dictionary values
def _get(d, *keys, default=None):
    cur = d
    for k in keys:
        if not isinstance(cur, dict) or k not in cur:
            return default
        cur = cur[k]
    return cur

# Function to generate header text for a game
def header_text(game):
    date = game.get("gameDate", "????-??-??")
    gid  = game.get("id", "?")

    away = _get(game, "awayTeam", "abbrev", default="AWAY")
    home = _get(game, "homeTeam", "abbrev", default="HOME")
    away_score = _get(game, "awayTeam", "score", default="?")
    home_score = _get(game, "homeTeam", "score", default="?")

    away_sog = _get(game, "awayTeam", "sog", default=_get(game, "awayTeam", "shotsOnGoal", default="?"))
    home_sog = _get(game, "homeTeam", "sog", default=_get(game, "homeTeam", "shotsOnGoal", default="?"))

    line1 = f"{date} | Game ID: {gid}"
    line2 = f"{away} @ {home}   Final: {away_score}-{home_score}   SoG: {away_sog}-{home_sog}"
    return line1, line2

In [16]:
# Create an integer slider widget for selecting game IDs
game_slider = widgets.IntSlider(min=1, max=len(game_ids), description="Game ID Index")

# Create an integer slider widget for selecting play indices
play_slider = widgets.IntSlider(min=1, max=len(games[0].get("plays", [])), step=1, description="Play Index", continuous_update=False)

In [17]:
output = widgets.Output()

In [18]:
# Function to synchronize play slider with selected game
def sync_play_slider():
    g = games[game_slider.value-1]
    plays = g.get("plays", [])
    play_slider.max = max(1, len(plays))
    play_slider.value = min(play_slider.value, play_slider.max)

# Function to draw the selected play on the hockey rink
def draw():
    g = games[game_slider.value-1]
    plays = g.get("plays", [])
    if not plays:
        with output:
            clear_output()
            print("Aucun play pour ce match.")
        return

    p = plays[play_slider.value-1]

    l1, l2 = header_text(g)
    title = play_title(p)

    if p.get("typeDescKey") == "goal":
        details = p.get("details") or {}
        scorer_id = details.get("scoringPlayerId")  # peut être dict/int/str
        scorer_name = get_player_name_cached(scorer_id)
        if scorer_name:
            title = f"{title} : {scorer_name}"


    xy = play_coordinates(p)  # ton helper
    with output:
        clear_output()

        fig, ax = plt.subplots(figsize=(10, 4))
        ax.imshow(rink_image, extent=[-110, 110, -50, 50], aspect="equal")

        if xy is not None:
            x, y = xy
            ax.scatter(
                [x], 
                [y], 
                s=80,
                c="blue",
                linewidths=2,
                zorder=5
                )

        ax.set_xlim(-100, 100)
        ax.set_ylim(-42.5, 42.5)
        ax.set_xlabel("feet")
        ax.set_ylabel("feet")

        # Texte au-dessus de l'image
        ax.set_title(f"{l1}\n{l2}\n{title}", fontsize=10)

        plt.show()

# Event handlers for slider changes
def on_game_change(change):
    sync_play_slider()
    draw()

# Event handlers for slider changes
def on_play_change(change):
    draw()

game_slider.observe(on_game_change, names="value")
play_slider.observe(on_play_change, names="value")

display(widgets.VBox([
    widgets.HBox([game_slider, play_slider]),
    output
]))

# Ajouter des boutons pour naviguer entre les jeux et les parties
btn_next_play = widgets.Button(description="Next play ▶", icon="step-forward")
btn_prev_play = widgets.Button(description="◀ Prev play")

btn_next_game = widgets.Button(description="Next game ▶▶", icon="forward")
btn_prev_game = widgets.Button(description="◀◀ Prev game")

def next_play(_):
    if play_slider.value < play_slider.max:
        play_slider.value += 1
    else:
        # option: passer au match suivant automatiquement
        if game_slider.value < game_slider.max:
            game_slider.value += 1
            # play_slider sera resync dans on_game_change()
            play_slider.value = 0

def prev_play(_):
    if play_slider.value > 0:
        play_slider.value -= 1
    else:
        # option: revenir au match précédent et aller au dernier play
        if game_slider.value > 0:
            game_slider.value -= 1
            # après resync, on met au dernier play
            play_slider.value = play_slider.max

def next_game(_):
    if game_slider.value < game_slider.max:
        game_slider.value += 1
        play_slider.value = 0

def prev_game(_):
    if game_slider.value > 0:
        game_slider.value -= 1
        play_slider.value = 0

btn_next_play.on_click(next_play)
btn_prev_play.on_click(prev_play)
btn_next_game.on_click(next_game)
btn_prev_game.on_click(prev_game)

controls = widgets.HBox([btn_prev_game, btn_prev_play, btn_next_play, btn_next_game])
display(controls)

sync_play_slider()
draw()